In [ ]:
# 🛠️ Install required libraries (only once in Colab)
!pip install langgraph transformers peft accelerate bitsandbytes torch torchvision
!pip install pytesseract pillow wikipedia

# 📦 Import all required packages
import pytesseract
import wikipedia
from PIL import Image
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Blip2Processor, Blip2ForConditionalGeneration
from peft import PeftModel
from langgraph.graph import StateGraph, END
from google.colab import files


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `botpc` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate wh

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration

model_id = "Salesforce/blip2-flan-t5-xl"

# Load processor
blip_processor = Blip2Processor.from_pretrained(model_id)

# Load model with float16 + auto device placement
blip_model = Blip2ForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)



Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/5.81G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
blip_processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
import zipfile
import os

zip_file = "qlora-chatbot-model.zip"  # name of your uploaded zip file
extract_dir = "qlora-chatbot-model"   # folder to extract into

# Unzip the file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# List extracted files
print("✅ Extracted to:", extract_dir)
print("📂 Files inside:", os.listdir(extract_dir))



✅ Extracted to: qlora-chatbot-model
📂 Files inside: ['qlora-chatbot-model']


In [ ]:
# 🎯 Load your fine-tuned LoRA adapter
model_path = "/content/qlora-chatbot-model/qlora-chatbot-model"


model = PeftModel.from_pretrained(base_model, model_path)

model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_pro

In [ ]:
#test-caption node


from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch
from PIL import Image


# Image caption function
def caption_image(image_path):
    if blip_model is None:
        print("Model not loaded correctly.")
        return

    raw_image = Image.open(image_path).convert('RGB')
    inputs = processor(images=raw_image, return_tensors="pt").to(blip_model.device)

    with torch.no_grad():
        generated_ids = blip_model.generate(**inputs)
        caption = processor.decode(generated_ids[0], skip_special_tokens=True)
        print("Caption:", caption)

# Example use
caption_image("header.jpg")


FileNotFoundError: [Errno 2] No such file or directory: 'header.jpg'

In [ ]:
from google.colab import files
import pytesseract
import wikipedia
from PIL import Image
import torch
from typing import TypedDict
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    BlipProcessor, BlipForConditionalGeneration
)
from peft import PeftModel
from langgraph.graph import StateGraph, END
from google.colab import files

In [ ]:
# ✅ 1. Define Typed State for LangGraph
class ChatState(TypedDict):
    image: Image.Image
    caption: str
    ocr: str
    web: str
    question: str
    answer: str





In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict
import torch
import wikipedia
import pytesseract
from PIL import Image
from transformers import AutoTokenizer, AutoModelForCausalLM, Blip2Processor, Blip2ForConditionalGeneration


# === Define State ===
class ChatState(TypedDict):
    image: Image.Image
    caption: str
    ocr: str
    web: str
    question: str
    answer: str


# === Load Models ==


# === Define Nodes ===
def caption_node(state: ChatState) -> ChatState:
    inputs = blip_processor(state["image"], return_tensors="pt").to(blip_model.device)
    with torch.no_grad():
        output = blip_model.generate(**inputs, max_new_tokens=50)  # Increase token limit

    state["caption"] = blip_processor.decode(output[0], skip_special_tokens=True)
    return state

def ocr_node(state: ChatState) -> ChatState:
    image = state["image"].convert("L")
    raw_text = pytesseract.image_to_string(image, config="--psm 6 --oem 3")
    lines = [line.strip() for line in raw_text.split("\n") if len(line.strip()) > 2]

    state["ocr"] = "\n".join(lines) if lines else ""
    return state




def web_node(state: ChatState) -> ChatState:
    try:
        caption_words = state["caption"].split()
        keywords = [w for w in caption_words if w.lower() not in {"with", "a", "the", "of", "and"}][:3]
        query = " ".join(keywords) if keywords else "game of thrones"

        summary = wikipedia.summary(query, sentences=2)
        state["web"] = summary
    except Exception:
        state["web"] = "No relevant web info found."
    return state




def qa_node(state: ChatState) -> ChatState:
    prompt = f"""You are an assistant that answers questions about what's in an image.

Image Caption: {state['caption']}
Text in Image (OCR): {state['ocr'] or 'No text found.'}
Relevant Info: {state['web'] or 'None'}

Now answer this clearly and briefly based on the actual content in the image:
Question: {state['question']}

Answer:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=80,
            temperature=0.6,
            top_p=0.9,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    state["answer"] = response.split("Answer:")[-1].strip() if "Answer:" in response else response.strip()
    return state


# === Build LangGraph ===
graph = StateGraph(ChatState)
graph.add_node("caption", caption_node)
graph.add_node("ocr", ocr_node)
graph.add_node("web", web_node)
graph.add_node("qa", qa_node)

graph.set_entry_point("caption")
graph.add_edge("caption", "ocr")
graph.add_edge("ocr", "web")
graph.add_edge("web", "qa")
graph.add_edge("qa", END)

app = graph.compile()


In [ ]:
from PIL import Image
from google.colab import files

while True:
    print("\n📤 Upload an image (or type 'exit' to quit):")
    uploaded = files.upload()
    if not uploaded:
        break

    image_path = list(uploaded.keys())[0]
    if "exit" in image_path.lower():
        break

    image = Image.open(image_path).convert("RGB")

    # Prepare initial state
    state: ChatState = {
        "image": image,
        "caption": "",
        "ocr": "",
        "web": "",
        "question": "",
        "answer": ""
    }

    # Build LangGraph once for caption → ocr → web
    temp_graph = StateGraph(ChatState)
    temp_graph.add_node("caption", caption_node)
    temp_graph.add_node("ocr", ocr_node)
    temp_graph.add_node("web", web_node)
    temp_graph.set_entry_point("caption")
    temp_graph.add_edge("caption", "ocr")
    temp_graph.add_edge("ocr", "web")
    temp_graph.add_edge("web", END)
    preprocess = temp_graph.compile()

    # Process image
    print("⚙️ Analyzing image...\n")
    state = preprocess.invoke(state)

    # Show state
    print("📊 STATE INFO:")
    print("🖼️ Caption:", state["caption"] or "[Empty]")
    print("🔠 OCR:", state["ocr"] or "[None found]")
    print("🌐 Web Info:", state["web"] or "[No info]")

    # Multiple question loop
    while True:
        question = input("\n❓ Ask about this image (or type 'next' or 'exit'): ").strip()
        if question.lower() in ["exit", "next"]:
            break

        state["question"] = question
        state["answer"] = ""

        print("🧠 Generating answer...")
        final_state = qa_node(state)
        print("🤖 Assistant:", final_state["answer"])



📤 Upload an image (or type 'exit' to quit):


Saving header.jpg to header (1).jpg
⚙️ Analyzing image...

📊 STATE INFO:
🖼️ Caption: red dead redemption 2 - pc game download
🔠 OCR: ” ey
ae ae
i303 9 ft
aie
🌐 Web Info: No relevant web info found.


KeyboardInterrupt: Interrupted by user

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict
import torch
import wikipedia
from PIL import Image
from transformers import AutoTokenizer, AutoModelForCausalLM, Blip2Processor, Blip2ForConditionalGeneration

# === Define State ===
class ChatState(TypedDict):
    image: Image.Image
    caption: str
    web: str
    question: str
    answer: str

# === Define Nodes ===

def caption_node(state: ChatState) -> ChatState:
    inputs = blip_processor(images=state["image"], return_tensors="pt").to(blip_model.device)
    with torch.no_grad():
        output = blip_model.generate(**inputs, max_new_tokens=50)
    state["caption"] = blip_processor.decode(output[0], skip_special_tokens=True)
    return state

def web_node(state: ChatState) -> ChatState:
    try:
        # Remove common stop words and take unique meaningful words
        caption = state["caption"].lower()
        stop_words = {"a", "an", "the", "with", "of", "in", "on", "and", "for", "to", "is"}
        words = [w.strip() for w in caption.split() if w.strip() and w.lower() not in stop_words and len(w) > 2]

        # Take up to 4 unique keywords
        keywords = list(dict.fromkeys(words))[:4]  # preserve order, remove duplicates

        query = " ".join(keywords)
        summary = wikipedia.summary(query, sentences=4)
        state["web"] = summary
    except:
        state["web"] = "No relevant web info found."
    return state



def qa_node(state: ChatState) -> ChatState:
    prompt = f"""You are an assistant that answers questions about what's in an image.

Image Caption: {state['caption']}
Relevant Info: {state['web'] or 'None'}

Now answer this clearly and briefly based on the actual content in the image:
Question: {state['question']}

Answer:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.6,
            top_p=0.9,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    state["answer"] = response.split("Answer:")[-1].strip() if "Answer:" in response else response.strip()
    return state

# === Build LangGraph ===
graph = StateGraph(ChatState)
graph.add_node("caption", caption_node)
graph.add_node("web", web_node)
graph.add_node("qa", qa_node)

graph.set_entry_point("caption")
graph.add_edge("caption", "web")
graph.add_edge("web", "qa")
graph.add_edge("qa", END)

app = graph.compile()

# === Runtime Loop for Colab or Notebook ===
from google.colab import files

while True:
    print("\n📤 Upload an image (or type 'exit' to quit):")
    uploaded = files.upload()
    if not uploaded:
        break

    image_path = list(uploaded.keys())[0]
    if "exit" in image_path.lower():
        break

    image = Image.open(image_path).convert("RGB")

    # Prepare initial state
    state: ChatState = {
        "image": image,
        "caption": "",
        "web": "",
        "question": "",
        "answer": ""
    }

    # Step 1: caption → web
    preprocess = StateGraph(ChatState)
    preprocess.add_node("caption", caption_node)
    preprocess.add_node("web", web_node)
    preprocess.set_entry_point("caption")
    preprocess.add_edge("caption", "web")
    preprocess.add_edge("web", END)
    run_preprocess = preprocess.compile()

    print("⚙️ Analyzing image...\n")
    state = run_preprocess.invoke(state)

    # Show state
    print("📊 STATE INFO:")
    print("🖼️ Caption:", state["caption"] or "[Empty]")
    print("🌐 Web Info:", state["web"] or "[No info]")

    # Q&A Loop
    while True:
        question = input("\n❓ Ask about this image (or type 'next' or 'exit'): ").strip()
        if question.lower() in ["exit", "next"]:
            break

        state["question"] = question
        state["answer"] = ""

        print("🧠 Generating answer...")
        final_state = qa_node(state)
        print("🤖 Assistant:", final_state["answer"])



📤 Upload an image (or type 'exit' to quit):


Saving stranger-things-4-volume-2.jpeg to stranger-things-4-volume-2.jpeg
⚙️ Analyzing image...

📊 STATE INFO:
🖼️ Caption: a poster for stranger things season 1 - stranger things season 1 poster
🌐 Web Info: No relevant web info found.


KeyboardInterrupt: Interrupted by user

In [ ]:
!pip install gradio --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.5/324.5 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 101.1 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.10.1
    Uninstalling gradio_client-1.10.1:
      Successfully uninstalled gradio_client-1.10.1
  Attempting uninstall: gradio
    Found existing installation: gradio 5.31.0
    Uninstalling gradio-5.31.0:
      Successfully uninstalled gradio-5.31.0


In [ ]:
import gradio as gr

# 🧠 Gradio wrapper function
def chat_with_image(image, question):
    try:
        if not image or not question.strip():
            return "<div style='color:red;font-weight:bold;'>⚠️ Please upload an image and type a question.</div>"

        image = image.convert("RGB")

        # Initial LangGraph state (OCR removed)
        state: ChatState = {
            "image": image,
            "caption": "",
            "web": "",
            "question": question,
            "answer": ""
        }

        # Run LangGraph app
        final_state = app.invoke(state)

        # HTML response
        return f"""
        <div style='font-family:JetBrains Mono, monospace; color:#00f7ff; font-size:16px; background:#121212; padding:15px; border-radius:10px'>
            <b style='color:#FFFFFF;'>🤖 Assistant:</b><br>{final_state['answer']}
        </div>
        """
    except Exception as e:
        return f"<div style='color:red;font-weight:bold;'>❌ Error: {str(e)}</div>"

# 🌌 Custom UI CSS
custom_css = """
body {
    background-color: #0d0d0d;
}
.gradio-container {
    font-family: 'JetBrains Mono', monospace;
    color: white;
}
textarea, input, button {
    font-size: 16px !important;
}
.gr-button {
    background: #00f7ff !important;
    color: #000 !important;
    font-weight: bold;
    border-radius: 6px;
    border: none;
}
.gr-button:hover {
    background: #00c4cc !important;
    transform: scale(1.03);
}
"""

# 🎨 UI Layout
with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("<h1 style='font-family:Michroma, sans-serif; color:#00f7ff;'>AION</h1>")
    gr.Markdown("<p style='color:#bbbbbb;'>Upload an image, ask a question, and get a smart AI answer using LangGraph + QLoRA + Vision AI.</p>")

    with gr.Row():
        image_input = gr.Image(type="pil", label="📸 Upload Image")
        question_input = gr.Textbox(label="❓ Your Question", placeholder="e.g. What is this device used for?")

    with gr.Row():
        ask_btn = gr.Button("💬 Ask AION")
        reset_btn = gr.Button("🔁 Reset")

    output = gr.HTML()

    ask_btn.click(chat_with_image, inputs=[image_input, question_input], outputs=output)
    reset_btn.click(fn=lambda: ("", "", ""), inputs=[], outputs=[image_input, question_input, output])

# 🚀 Launch
demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dced3d7fe4dcd98930.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
